In [1]:
#pip install youtube-transcript-api
#pip install pytube 

In [2]:
from langchain.document_loaders import YoutubeLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain

In [3]:
# đây là câu lệnh nối biến giữa các file 
# tại file có tên là secret_key, nó chứa một câu lệnh là openai_api_key = " API của openai"
import os
import openai
import sys
sys.path.append('../..')

from secret_key import openai_api_key

os.environ["OPENAI_API_KEY"] = openai_api_key

# 1. Simple video( short videos ) 
dành cho các video có thời lượng ngắn 


In [6]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=ahVe_nyboNg" , add_video_info=True)


In [8]:
result = loader.load ()

In [37]:
print (type( result))
print (f"Found video from {result[0].metadata['author']} that is {result[0].metadata['length']} seconds long")
print ("")
print (result)


<class 'list'>
Found video from CNN that is 313 seconds long

[Document(page_content=">> CHRIS CHRISTIE TRENDING THE CAMPAIGN TRAIL FOR WARS AND REPUBLICAN PRESIDENTIAL CANDIDATE MET WITH UKRAINIAN PRESIDENT ZELINSKI DURING AN UNANNOUNCED TRIP TO THE WAR-TORN COUNTRY TODAY FOR THE FORMER NEW JERSEY GOVERNOR VISITED BATTLE SCARRED AREAS. HE IS NOW THE SECOND GOP CANDIDATE TO VISIT UKRAINE AND MEETS HONESTY. MIKE PENCE OBVIOUSLY WAS THERE IN JUNE. THEIR SUPPORT UP SUITS IN CONTRAST TO SEVERAL OF THEIR RIVALS. INCLUDING DONALD TRUMP THE FRONT RUNNER. REFUSED TO SAY WHO HE WANTED TO WIN THE WAR. RUSSIA OR UKRAINE. GOVERNOR RHONDA SANTOS WAS HAD TO WALK BACK HIS PRIOR CLAIM THAT IT WAS A TERRITORIAL DISPUTE. >> I'M JOINED NOW BY CNN'S SENIOR POLITICAL COMMENTATOR AT KENSINGTON 10 FORMER REPUBLICAN CONGRESSMAN, ILLINOIS CONGRESSMAN. THANKS FOR JOINING ME TODAY FOR 71 PERCENT OF YOUR PARTY REPUBLICANS SAY THAT CONGRESS SHOULD NOT AUTHORIZE THE FUNDING. 59 PERCENT BASED ON THE IMPULSE SAY THAT

In [9]:
llm = OpenAI(temperature=0, openai_api_key = "OPENAI_API_KEY" ) # để chạy câu lệnh này thì cần có key đc trả phí 

In [33]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

#  Long videos 
dành cho các video có thời lượng và script dài, chúng ta cần chia nhỏ đoạn script của video thành nhiều phần để có thể tối ưu được token của Chat GPT 

In [34]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=ahVe_nyboNg" , add_video_info=True)

In [35]:
result = loader.load()

In [36]:
# câu lệnh này nhằm mục đích summarize là script của video 
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)


AuthenticationError: Incorrect API key provided: OPENAI_A**_KEY. You can find your API key at https://platform.openai.com/account/api-keys.

Problem, transcript/document is too long 

In [6]:
#from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
# câu lệnh sau dùng để phân tách script của video trở nên ngắn hơn. 
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter =  RecursiveCharacterTextSplitter ( chunk_size=2000, chunk_overlap=0 )
# chunk_size: là số lượng chữ của mỗi đoạn văn được phân tách
# chunk_overlap=0 có nghĩa là không có sự chồng chéo đoạn văn trong quá trình phân tách 
texts = text_splitter.split_documents(result)

In [8]:
# số lượng đoạn văn được phân tách ra 
len(texts)

3

In [ ]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True ) # summarize đoạn scripts cảu youtube 
chain.run(texts[:4])

# Multiple videos 

In [81]:
youtube_url_list = ["https://www.youtube.com/watch?v=pNcQ5XXMgH4&list=PLqZXAkvF1bPNQER9mLmDbntNfSpzdDIU5&index=9",
                    "https://www.youtube.com/watch?v=ahVe_nyboNg"]

texts = []

text_splitter =  RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0 )

for url in youtube_url_list : 
    loader = YoutubeLoader.from_youtube_url(url, add_video_info=True)
    result = loader.load()
    
    texts.extend(text_splitter.split_documents(result))

    

In [82]:
len(texts)


10

In [ ]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True )
chain.run(texts)

# PDFs
need pip install pypdf

In [ ]:
import os
import openai
import sys
sys.path.append('../..')


from secret_key import openai_api_key

os.environ["OPENAI_API_KEY"] = openai_api_key

In [45]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [46]:
# đưa tài liệu vào 
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("MachineLearning-Lecture01.pdf")
pages = loader.load()

In [47]:
len(pages) # số trang 

22

In [48]:
page = pages[0]

In [49]:
print(page.page_content[0:1000]) # in ra nội dung của văn bản, giới hạn từ là 1000 từ v

MachineLearning-Lecture01  
Instructor (Andrew Ng):  Okay. Good morning. Welcome to CS229, the machine 
learning class. So what I wanna do today is ju st spend a little time going over the logistics 
of the class, and then we'll start to  talk a bit about machine learning.  
By way of introduction, my name's  Andrew Ng and I'll be instru ctor for this class. And so 
I personally work in machine learning, and I' ve worked on it for about 15 years now, and 
I actually think that machine learning is th e most exciting field of all the computer 
sciences. So I'm actually always excited about  teaching this class. Sometimes I actually 
think that machine learning is not only the most exciting thin g in computer science, but 
the most exciting thing in all of human e ndeavor, so maybe a little bias there.  
I also want to introduce the TAs, who are all graduate students doing research in or 
related to the machine learni ng and all aspects of machin e learning. Paul Baumstarck 
works in mach

In [44]:
page.metadata # thông tin liên quan của tài liệu 

{'source': 'MachineLearning-Lecture01.pdf', 'page': 0}